# Portfolio VaR Notebook

This notebook computes portfolio Value at Risk (VaR) using historical prices from yfinance. It supports configurable portfolio constituents, confidence level, and returns timeframe (daily/weekly/monthly). Methodology: adjusted close prices → resampled → log returns → portfolio P&L → Historical VaR (and optional parametric VaR for comparison). Limitations: relies on price history quality and assumes stationarity for historical VaR; parametric results assume approximate normality.


In [ ]:
# Setup
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from src.portfolio import Asset, Portfolio, validate_confidence_level, validate_timeframe
from src.data import fetch_adjusted_close, align_on_common_dates, coverage_table
from src.returns import resample_prices, compute_log_returns, latest_prices_from_frame, portfolio_pnl_series
from src.var import summarize_var
from src.plotting import plot_prices, plot_portfolio_pnl, plot_return_histogram

pd.options.display.float_format = lambda x: f"{x:,.6f}"


In [ ]:
# Portfolio configuration (edit and re-run to change)
confidence = 0.99  # 0.90–0.9999
timeframe = "daily"  # one of: daily, weekly, monthly
start_date = None  # e.g., "2018-01-01" or None
end_date = None    # e.g., "2024-12-31" or None

portfolio = Portfolio([
    Asset(ticker="AAPL", name="Apple", position=10),
    Asset(ticker="MSFT", name="Microsoft", position=8),
    Asset(ticker="TSLA", name="Tesla", position=5),
])

# Validate inputs
validate_confidence_level(confidence)
timeframe = validate_timeframe(timeframe)

print("Recreate this portfolio by running:\n")
print(portfolio.recreate_snippet())


In [ ]:
# Data retrieval
prices_raw = fetch_adjusted_close(portfolio.tickers(), start=start_date, end=end_date, progress=False)

if prices_raw.empty:
    raise RuntimeError("No price data fetched. Check tickers/date range/network.")

# Alignment and coverage
prices_aligned, dropped = align_on_common_dates(prices_raw, min_obs=60)

if dropped:
    print(f"Warning: Dropped tickers with insufficient observations: {', '.join(dropped)}")

coverage = coverage_table(prices_raw)
print("Data coverage (raw):")
coverage


In [ ]:
# Returns and exposures
prices_resampled = resample_prices(prices_aligned, timeframe)
asset_returns = compute_log_returns(prices_resampled)

# Latest prices and holdings table
latest_px = prices_resampled.ffill().iloc[-1]
holdings = portfolio.holdings_table(latest_px)

# Per-asset return stats
from src.returns import returns_statistics
ret_stats = returns_statistics(asset_returns)

holdings, ret_stats


In [ ]:
# Portfolio P&L series (currency)
from src.returns import portfolio_pnl_series

pnl_series = portfolio_pnl_series(asset_returns, prices_resampled, portfolio.positions_series())
portfolio_value = portfolio.current_value(latest_px)

print(f"Portfolio current value: {portfolio_value:,.2f}")
pnl_series.tail()


In [ ]:
# VaR and contributions
result, contrib = summarize_var(
    pnl_series=pnl_series,
    portfolio_value=portfolio_value,
    confidence=confidence,
    timeframe=timeframe,
    asset_returns=asset_returns,
    positions=portfolio.positions_series(),
    latest_prices=latest_px,
)

summary = pd.DataFrame([
    {
        "Confidence": f"{result.confidence:.2%}",
        "Timeframe": result.timeframe,
        "Hist VaR (abs)": result.historical_var_abs,
        "Hist VaR (%PV)": result.historical_var_pct,
        "Param VaR (abs)": result.parametric_var_abs,
        "Param VaR (%PV)": result.parametric_var_pct,
    }
])
summary


In [ ]:
# Contributions table (if available)
if contrib is not None and not contrib.empty:
    contrib_df = contrib.to_frame(name="Contribution (currency)")
    contrib_df = contrib_df.sort_values(by=contrib_df.columns[0], ascending=False)
    contrib_df
else:
    print("Contribution approximation unavailable (insufficient data).")


In [ ]:
# Plots
_ = plot_prices(prices_resampled, names_map=portfolio.names_map())
_ = plot_portfolio_pnl(pnl_series)
_ = plot_return_histogram(pnl_series, var_abs=result.historical_var_abs, confidence=confidence)


## Summary
- Portfolio holdings and data coverage are shown above.
- Historical VaR is computed from the left-tail quantile of portfolio P&L; positive value indicates estimated loss at the chosen confidence.
- Contributions are approximated via a parametric Euler allocation to provide a sense of marginal risk by asset.
- Changing the portfolio, timeframe, or confidence and re-running will update all outputs.
